In [ ]:
%pylab inline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import visibility as vis

## Generate some 1D data

In [ ]:
x_line = np.arange(0, 8*np.pi, 0.1)
y_line = 1 + np.sin(x_line) * np.sin(0.12*x_line)
# plot(x_line, y_line)

### Find visible part in a single pass through

In [ ]:
curr_max = 0
visible_part = []
for i, y in enumerate(y_line):
    if y > curr_max:
#         print(i, y)
        curr_max = y
        visible_part.append(i)
visible_part
figure()
plot(x_line, y_line)
plot(x_line[visible_part], y_line[visible_part], 'r+')

## Generate some data in polar grid

In [ ]:
d_r = 1
d_alpha = 30
r_surf = np.arange(0, 10, d_r)
alpha_surf = np.arange(0, 360, d_alpha)
xx1, xx2 = np.meshgrid(alpha_surf, r_surf)

y_surf = 1 + np.sin(xx1) +  np.sin(2*xx2) * np.sin(0.3*xx1+xx2)

### Compose the visible mask in a single iteration along radius

At the same time, save also the indices of maximum radius visible in the given angle (max_i_r)

In [ ]:
visible_mask, max_i_r = vis.find_visible_area(y_surf)
figure(figsize=(10,6))
plt.pcolormesh(xx1, xx2, y_surf, shading='flat')
# plt.scatter(xx1+0.5*d_alpha, xx2+0.5*d_r, c=visible_mask, cmap='jet')
plt.scatter(xx1[visible_mask]+0.5*d_alpha, xx2[visible_mask]+0.5*d_r, c='r')
plt.plot(alpha_surf+0.5*d_alpha, r_surf[max_i_r]+0.5*d_r, '--r')

In [ ]:
figure(figsize=(12,6))
for i_r in range(len(r_surf)):
    plot(alpha_surf, y_surf[i_r, :], label=r_surf[i_r])
plt.legend()
plot(xx1[visible_mask], y_surf[visible_mask], 'ok')
# for i_r, vis_mask in visible_part:
#     plot(alpha_surf[vis_mask], y_surf[i_r, vis_mask], 'ok')
plot(alpha_surf, y_surf[max_i_r, range(len(alpha_surf))], 'r--')

In [ ]:
def image_plot_grid(x, y, z, regular=True, indexing='xy'):
    dx = np.mean(np.diff(x))
    dy = np.mean(np.diff(y))
    if regular:
#         if indexing == 'xy':
#             origin='bottom'
#         elif indexing == 'ij':
#             origin=
        print((min(x)-dx/2, max(x)+dx/2, min(y)-dy/2, max(y)+dy/2))
        return plt.imshow(z, origin='bottom', extent = (min(x)-dx/2, max(x)+dx/2, min(y)-dy/2, max(y)+dy/2))
    else:
        raise(NotImplementedError)

## Generate cartesian 'altitude' data

In [ ]:
# Generate cartesian data
dx = 0.6
dy = 0.3
x = np.arange(0, 40, dx)
y = np.arange(0, 30, dy)
xx, yy = np.meshgrid(x, y)#, indexing='ij')
Z_cart = 1 + np.sin(xx) +  np.sin(2*yy) * np.sin(0.3*xx+yy)

# Select the observer location
obs_x = 8.5
obs_y = 15.

## Interpolate the 'altitude' data from cartesian to polar grid and calculate visibility

In [ ]:
r_polar, alpha_polar, x_polar, y_polar = vis.generate_polar_grid(obs_x, obs_y, x, y, dx=dx, dy=dy, size='standard')
Z_polar = vis.interpolate_from_grid_to_any_points(x, y, np.transpose(Z_cart), x_polar, y_polar)
mask_polar, max_i_r = vis.find_visible_area(Z_polar)

horizon_x = x_polar[max_i_r, range(len(alpha_polar))]
horizon_y = y_polar[max_i_r, range(len(alpha_polar))]
horizon_alpha = alpha_polar
horizon_r = r_polar[max_i_r]
horizon_Z = Z_polar[max_i_r, range(len(alpha_polar))]

In [ ]:
figure(figsize=(10,10))
# plt.pcolormesh(xx, yy, Z_cart, shading='flat')
image_plot_grid(x, y, Z_cart)
plt.plot(obs_x, obs_y, '+')
plt.scatter(x_polar, y_polar, s=40, c='k')
plt.scatter(x_polar[mask_polar], y_polar[mask_polar], c='r', s=60)
plt.plot(horizon_x, horizon_y, '--r')
plt.scatter(x_polar, y_polar, s=30, c=Z_polar)
plt.gca().set_aspect('equal')

In [ ]:
%pylab qt

In [ ]:
alpha_mesh, r_mesh = np.meshgrid(alpha_polar, r_polar)
figure(figsize=(10,6))
image_plot_grid(alpha_polar, r_polar, Z_polar)
# plt.pcolormesh(alpha_mesh, r_mesh, Z_polar, shading='flat')
# plt.scatter(alpha_mesh[mask_polar]+0.5*d_alpha, r_mesh[mask_polar]+0.5*d_r, c='r')
# plt.plot(horizon_alpha+0.5*d_alpha, horizon_r+0.5*d_r, '--r')
plt.scatter(alpha_mesh[mask_polar], r_mesh[mask_polar], c='r')
plt.plot(horizon_alpha, horizon_r, '--r')

In [ ]:
figure(figsize=(12,6))
for i_r, r_r in enumerate(r_polar):
    plot(alpha_polar, Z_polar[i_r, :], label=r_r)
plt.legend()
plot(alpha_mesh[mask_polar], Z_polar[mask_polar], 'ok')
plot(horizon_alpha, horizon_Z, 'r--')

## Interpolate the mask in polar grid back to cartesian (broken)

In [ ]:
mask_cart = np.zeros_like(Z_cart, dtype=int)
count_cart = np.zeros_like(Z_cart, dtype=int)
polar_cart_i = np.round(x_polar.reshape(-1) - x[0] / dx).astype(int)
polar_cart_j = np.round(y_polar.reshape(-1) - y[0] / dy).astype(int)
for i, j, mask_ij in zip(polar_cart_i, polar_cart_j, mask_polar.reshape(-1)):
    if i >= 0 and i < len(x) and j >= 0 and j < len(y):
        count_cart[j,i] += 1
        mask_cart[j,i] += mask_ij
mask_cart = (mask_cart / count_cart) > 0.1
mask_cart
Z_cart[~mask_cart] = -2

figure(figsize=(10,10))
# plt.pcolormesh(xx, yy, Z_cart, shading='flat')
image_plot_grid(x, y, Z_cart)
plt.plot(obs_x, obs_y, '+')
# plt.scatter(x_polar, y_polar, s=40, c='k')
plt.scatter(x_polar[mask_polar], y_polar[mask_polar], c='r', s=60)
plt.plot(x_polar[max_i_r, range(len(alpha_polar))], y_polar[max_i_r, range(len(alpha_polar))], '--r')
# plt.scatter(x_polar, y_polar, s=30, c=Z_polar)
plt.gca().set_aspect('equal')